In [2]:
from glob import glob
import matplotlib.pyplot as plt
import fileinput
import collections
import re
from gensim.models import KeyedVectors, WordEmbeddingSimilarityIndex, TfidfModel
from gensim.similarities import MatrixSimilarity, SparseTermSimilarityMatrix, SoftCosineSimilarity
from gensim.corpora import Dictionary
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

partie = [
    'KO',
    'Konfederacja',
    'Lewica',
    'niez',
    'PiS',
    'PSL-Kukiz15'
]

In [3]:
stopwords = open("polish.stopwords.txt", encoding='utf-8').read().splitlines()

documents = []
tags = []

for p in partie:
    wypowiedzi_partii = ''
    for posl in glob('partie/' + p + '/*'):
        wypowiedzi = glob(posl + '/*')

        for wypowiedz in wypowiedzi:
            tags.append(wypowiedz[len('partie/'):-len('.txt')].replace('\\',': '))
            wyp_text = ''
            with fileinput.input(wypowiedz, openhook=fileinput.hook_encoded("windows-1250")) as wyp:
                for line in wyp:
                    wyp_text += line
            
            wyp_text = wyp_text.replace('\n', ' ')
            wyp_text = re.sub('[,.:;!?()]', '', wyp_text)
            documents.append(
                [word for word in wyp_text.lower().split() if word not in stopwords])

In [4]:
wv = KeyedVectors.load("word2vec_100_3_polish.bin")
dictionary = Dictionary(documents)
tfidf = TfidfModel(dictionary=dictionary)

In [5]:
similarity_index = WordEmbeddingSimilarityIndex(wv)
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

In [6]:
index = SoftCosineSimilarity(tfidf[[dictionary.doc2bow(document) for document in documents]], similarity_matrix)

index.save('soft_cosine.index')

In [7]:
index = SoftCosineSimilarity.load('soft_cosine.index')

In [51]:
try:
    # query = input("Query: ").lower().split()
    query = "".lower().split()
    bow = dictionary.doc2bow(query, return_missing=True)
    query = tfidf[bow[0]]
    print("Słowa nierozpoznane: ", bow[1])
    similarities = index[query]
    for score, wyp in sorted(zip(similarities, tags), key= lambda t: t[0], reverse=True)[:20]:
        print('{:.3f} : {}'.format(score, wyp))
except Exception as e:
    print(e)

Słowa nierozpoznane:  {'są': 1, 'w': 1}
0.916 : PiS: Ozdoba Jacek: 19-12-2019
0.854 : Konfederacja: Berkowicz Konrad: 19-12-2019
0.802 : Lewica: Śmiszek Krzysztof: 19-12-2019
0.785 : Lewica: Nowicka Wanda: 24-07-2020
0.778 : KO: Kierwiński Marcin: 06-04-2020
0.775 : Lewica: Czarzasty Włodzimierz: 04-06-2020
0.773 : KO: Jachira Klaudia: 14-08-2020
0.763 : KO: Piątkowski Krzysztof: 12-02-2020
0.757 : Lewica: Maciejewska Beata: 21-11-2019
0.757 : Lewica: Gosek-Popiołek Daria: 06-05-2020
0.632 : KO: Jaśkowiak Joanna: 09-01-2020
0.632 : PiS: Płonka Elżbieta: 24-07-2020
0.623 : Konfederacja: Braun Grzegorz: 30-04-2020
0.615 : Konfederacja: Berkowicz Konrad: 23-01-2020
0.614 : PiS: Ast Marek: 09-01-2020
0.613 : KO: Kropiwnicki Robert: 20-12-2019
0.613 : Lewica: Żukowska Anna Maria: 12-02-2020
0.609 : KO: Piekarska Katarzyna Maria: 19-11-2019
0.609 : Konfederacja: Kamiński Krystian: 20-12-2019
0.609 : Konfederacja: Korwin-Mikke Janusz: 19-12-2019
